In [1]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

#Load environment variables from .env file 
_ : bool = load_dotenv(find_dotenv())

client = OpenAI = OpenAI



In [2]:
# Import necessary libraries
import os
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

# Load environment variables from .env file
_ : bool = load_dotenv(find_dotenv())
client : OpenAI = OpenAI()

# Example dummy function for animal information
def get_animal_info(animal: str) -> str:
    """Get information about a specific animal"""
    # Define information about different animals
    animal_info = {
        "dog": "Dogs are loyal companions.",
        "cat": "Cats are independent and make great pets.",
        "bird": "Birds come in various species and are known for their songs.",
    }
    # Return information about the specified animal or "Unknown animal" if not found
    return json.dumps({"animal": animal, "info": animal_info.get(animal, "Unknown animal")})

from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion


def run_animal_conversation(main_request: str) -> str:
    # Step 1: Create a list of messages, starting with the user's prompt
    messages = [{"role": "user", "content": main_request}]
    
    # Step 1.1: Define a tool (function) that the model can use during the conversation
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_animal_info",
                "description": "Get information about a specific animal",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "animal": {
                            "type": "string",
                            "description": "The name of the animal",
                        },
                    },
                    "required": ["animal"],
                },
            },
        }
    ]

    # First Request
    # Step 2: Send the user's prompt and available functions to the model
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is the default, but we'll be explicit
    )
    response_message: ChatCompletionMessage = response.choices[0].message
    
    # Display the assistant's first response
    print("* First Response: ", dict(response_message))

    tool_calls = response_message.tool_calls
    
    # Display tool calls made by the assistant in the first response
    print("* First Reponse Tool Calls: ", list(tool_calls))

    # Step 2.1: Check if the model wanted to call a function
    if tool_calls:
        # Step 3: Call the function
        available_functions = {
            "get_animal_info": get_animal_info,
        }

        messages.append(response_message)  # Extend conversation with the assistant's reply

        # Step 4: Send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            
            # Call the function and get its response
            function_response = function_to_call(
                animal=function_args.get("animal"),
            )
            
            # Extend conversation with the function response
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  

        # Display the messages sent in the second request
        print("* Second Request Messages: ", list(messages))
        
        # Step 5: Get a new response from the model where it can see the function response
        second_response: ChatCompletion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )
        
        # Display the assistant's second response
        print("* Second Response: ", dict(second_response))
        return second_response.choices[0].message.content

# Example usage
user_question = "Tell me about dogs."
result = run_animal_conversation(user_question)
print("Final Result:", result)


* First Response:  {'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [ChatCompletionMessageToolCall(id='call_iwWszGrrAswHrDhAM78Gf4YV', function=Function(arguments='{"animal": "dog"}', name='get_animal_info'), type='function')]}
* First Reponse Tool Calls:  [ChatCompletionMessageToolCall(id='call_iwWszGrrAswHrDhAM78Gf4YV', function=Function(arguments='{"animal": "dog"}', name='get_animal_info'), type='function')]
* Second Request Messages:  [{'role': 'user', 'content': 'Tell me about dogs.'}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_iwWszGrrAswHrDhAM78Gf4YV', function=Function(arguments='{"animal": "dog"}', name='get_animal_info'), type='function')]), {'tool_call_id': 'call_iwWszGrrAswHrDhAM78Gf4YV', 'role': 'tool', 'name': 'get_animal_info', 'content': '{"animal": "dog", "info": "Dogs are loyal companions."}'}]
* Second Response:  {'id': 'chatcmpl-8i0kz9u9ggOxUXlpQFy2f4EYaUyxA

In [31]:


from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
_ : bool = load_dotenv (find_dotenv())

client : OpenAI = OpenAI()

# Import the 'json' module for working with JSON data
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API

def get_current_weather(location: str, unit: str = "fahrenheit") -> str:
    """Get the current weather in a given location"""
    
    print("Location:" , location)
    
    # Convert the location to lowercase for case-insensitive comparison
    location_lower = location.lower()

    # Check if the location is "tokyo"
    if "tokyo" in location_lower:
        # If it's Tokyo, return JSON-encoded weather data for Tokyo
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    
    # Check if the location is "san francisco"
    elif "san francisco" in location_lower:
        # If it's San Francisco, return JSON-encoded weather data for San Francisco
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    
    # Check if the location is "paris"
    elif "paris" in location_lower:
        # If it's Paris, return JSON-encoded weather data for Paris
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    
    # If the location doesn't match any predefined locations
    else:
        # Return JSON-encoded weather data with "unknown" temperature for the provided location
        return json.dumps({"location": location, "temperature": "unknown"})

    


In [32]:
from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion
# from openai.types.chat.chat_completion import ChatCompletionMessageParam, ChatCompletionMessageParam

def run_conversation(main_request: str)->str:
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": main_request}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]

    # First Request
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message: ChatCompletionMessage = response.choices[0].message
    display("* First Response: ", dict(response_message))

    tool_calls = response_message.tool_calls
    display("* First Reponse Tool Calls: ", list(tool_calls))

    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        
        messages.append(response_message)  # extend conversation with assistant's reply
        
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        display("* Second Request Messages: ", list(messages))
        second_response: ChatCompletion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print("* Second Response: ", dict(second_response))
        return second_response.choices[0].message.content

In [33]:
run_conversation("what is the current weather in San Francisco, Tokyo and Paris ?")

'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_9VMKdobNJqb7V2t4o2en0qxz', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_ruvEg8j1Uuea2jq1anei8Azw', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_fBZ4MOGboeCQrln97x8ZPVhz', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]}

'* First Reponse Tool Calls: '

[ChatCompletionMessageToolCall(id='call_9VMKdobNJqb7V2t4o2en0qxz', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_ruvEg8j1Uuea2jq1anei8Azw', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_fBZ4MOGboeCQrln97x8ZPVhz', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]

Location: San Francisco
Location: Tokyo
Location: Paris


'* Second Request Messages: '

[{'role': 'user',
  'content': 'what is the current weather in San Francisco, Tokyo and Paris ?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9VMKdobNJqb7V2t4o2en0qxz', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_ruvEg8j1Uuea2jq1anei8Azw', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_fBZ4MOGboeCQrln97x8ZPVhz', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_9VMKdobNJqb7V2t4o2en0qxz',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"}'},
 {'tool_call_id': 'call_ruvEg8j1Uuea2jq1anei8Azw',
  'rol

* Second Response:  {'id': 'chatcmpl-8hbtsEkR2IcQyXOW2Ry8tFzSp0cy8', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current weather in San Francisco is 72°F, in Tokyo it is 10°C, and in Paris it is 22°C.', role='assistant', function_call=None, tool_calls=None))], 'created': 1705404484, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_cbe4fa03fe', 'usage': CompletionUsage(completion_tokens=28, prompt_tokens=168, total_tokens=196)}


'The current weather in San Francisco is 72°F, in Tokyo it is 10°C, and in Paris it is 22°C.'

<image src = "https://raw.githubusercontent.com/panaverse/learn-generative-ai/28b9b98f626f9bea460bd81b86aea3e0355332fe/03_chatgpt/05_function_calling/second.png">

In [60]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

#Load environment variables from .env file 
_ : bool = load_dotenv(find_dotenv())

client = OpenAI = OpenAI()

def weather_function (location : str, unit : str = "fahrenheit")->str:
    if "Tokyo" in location:
        return json.dumps ({"location": "Tokyo", "temperature": 22, "unit": "celicius"})
    elif "California" in location:
        return json.dumps({"location": "California", "temperature": 55, "unit": "fahrenheit"})
    elif "San Francisco" in location:
        return json.dumps ({"location": "San Francisco", "tempearature": 75, "unit": "celcius"})
    else:
        return json.dumps({"location": location, "temperature": "I'm sorry I can't tell you"})
    
from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion

def conversation (user_request : str)->str:

    messages = [{"role": "user", "content" : user_request}]

    tools = [
        {
            "type": "function",
            "function":{
                "name": "weather_function",
                "description": "get the current weathher of the given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type":"string",
                            "description": "Name of the location or the state like California or San Francisco et cetra",
                        },
                        "unit": {"type": "string", "enum":["celcius","fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]

    response : ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto", #auto is default, like to make the function run, we will write Auto, if you write none the function wom't run 
    )

    response_message: ChatCompletion = response.choices[0].message   # reponse_message kai naam se variable create kar kai pehla response print karwaya 
    print("First Response:", dict(response_message))

    tool_calls = response_message.tool_calls                      #tool_calls ka variable create kiya, us ko first response se connect kiya and pehla tool call ka respone print krwaya 
    print("* First response tool calls:", list(tool_calls))

    if tool_calls:
        available_functions = {
            "weather_function": weather_function,
        }

        messages.append(response_message)

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location = function_args.get("location"),
                unit = function_args.get("unit"),
            )

            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )

     
            display("Second Request Message:", list(messages))
            second_response: ChatCompletion = client.chat.completions.create(
                model = "gpt-3.5-turbo-1106",
                messages = messages,
            )

            print ("Second Response:", dict(second_response))
            return second_response.choices[0].message.content


In [61]:
conversation("What is the weather in California")


First Response: {'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [ChatCompletionMessageToolCall(id='call_rdfWrujdmvon8kcgTxWDCsbl', function=Function(arguments='{"location": "California", "unit": "celcius"}', name='weather_function'), type='function')]}
* First response tool calls: [ChatCompletionMessageToolCall(id='call_rdfWrujdmvon8kcgTxWDCsbl', function=Function(arguments='{"location": "California", "unit": "celcius"}', name='weather_function'), type='function')]


'Second Request Message:'

[{'role': 'user', 'content': 'What is the weather in California'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_rdfWrujdmvon8kcgTxWDCsbl', function=Function(arguments='{"location": "California", "unit": "celcius"}', name='weather_function'), type='function')]),
 {'tool_call_id': 'call_rdfWrujdmvon8kcgTxWDCsbl',
  'role': 'tool',
  'name': 'weather_function',
  'content': '{"location": "California", "temperature": 55, "unit": "fahrenheit"}'}]

Second Response: {'id': 'chatcmpl-8iaZSOAxUJsQ0pMYVgtM5Y8OoJ0Ds', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current temperature in California is 55°F.', role='assistant', function_call=None, tool_calls=None))], 'created': 1705637702, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_c596c86df9', 'usage': CompletionUsage(completion_tokens=10, prompt_tokens=60, total_tokens=70)}


'The current temperature in California is 55°F.'